# Question 3 Performance Analystics

input: 
- price time series of AAPL and YSLA
- corresponding transactions of purchases and divestments of the 2 securities from Jan 2011 to 31 Dec 2020

The Excel file provides two sheets:
1. Q3 transaction: trades executed with data, direcion, transactions, and amount of money spent in the transaction recorded. 
2. Q3 price: daily closing prices for TSLA and AAPL

## Data Cleaning 

In [ ]:
import pandas as pd

# load the Excel file
excel_path = "./PA_case_study.xlsx"
xls = pd.ExcelFile(excel_path)

# load the two sheets
transactions_df = xls.parse("Q3 Transaction")
prices_df = xls.parse("Q3 Price")

## clean the prices dataframe
# Drop empty or malformed rows
prices_df = prices_df.dropna()

# convert Date to datetime format
prices_df['Date'] = pd.to_datetime(prices_df['Date'])

# rename price columns for simplicity
prices_df.rename(columns={"TSLA-US": "TSLA", "AAPL-US": "AAPL"}, inplace=True)

## clean the transaction dataframe
# convert Date column to datetime
transactions_df['Date'] = pd.to_datetime(transactions_df['Date'])

# ensure the US$ column is numeric
transactions_df = transactions_df.copy()
transactions_df['(US$)'] = pd.to_numeric(transactions_df['(US$)'], errors='coerce')

# drop rows with missing transaction amounts
transactions_df = transactions_df.dropna(subset=['(US$)'])

# rename columns for clarity
transactions_df.columns = ['Date', 'Direction', 'Ticker', 'Amount_USD']

# display clean DataFrames
display(transactions_df.head())
display(prices_df.head())
# len(prices_df)


,Date,Direction,Ticker,Amount_USD
0,2011-01-03,BUY,TSLA,1000000
1,2011-01-03,BUY,AAPL,1000000
2,2012-01-03,BUY,AAPL,1000000
3,2019-04-01,SELL,TSLA,9000000
4,2020-03-23,SELL,TSLA,2000000


,Date,TSLA,AAPL
2,2011-01-03,5.324001,11.770358
3,2011-01-04,5.334000,11.831786
4,2011-01-05,5.366001,11.928572
5,2011-01-06,5.576000,11.918929
6,2011-01-07,5.648000,12.004286


2517

## Task a: Calculate IRRs, Annualized Time Weighted Return and TVPI

In [28]:
def calculate_irr(cashflows, dates, max_iterations=1000, tolerance=1e-6):
    """
    calculate Internal Rate of Return using Newton-Raphson method
    
    Args:
        cashflows: List of cash flows (negative for investments, positive for returns)
        dates: List of corresponding dates
        max_iterations: Maximum number of iterations
        tolerance: Convergence tolerance
    
    Returns:
        IRR as decimal 
    """
    if len(cashflows) != len(dates):
        raise ValueError("Cashflows and dates must have same length")
    
    # convert dates to days from first date for calculation
    days = [(d - dates[0]).days for d in dates]
    
    # initial guess for IRR
    irr = 0.1  # 10%
    
    for i in range(max_iterations):
        # calculate NPV and its derivative
        npv = sum(cf / (1 + irr) ** (day/365.25) for cf, day in zip(cashflows, days))
        d_npv = sum(-cf * (day/365.25) / (1 + irr) ** ((day/365.25) + 1) for cf, day in zip(cashflows, days))
        
        if abs(npv) < tolerance:
            return irr
        
        if abs(d_npv) < tolerance:
            break
            
        # Newton-Raphson update
        irr_new = irr - npv / d_npv
        
        if abs(irr_new - irr) < tolerance:
            return irr_new
            
        irr = irr_new
    
    return irr

def calculate_performance_metrics(ticker, transactions_df, prices_df):
    """
    calculate IRR, Time-Weighted Return, and TVPI for a specific ticker
    
    Args:
        ticker: Stock ticker (e.g., 'AAPL', 'TSLA')
        transactions_df: DataFrame with transaction data
        prices_df: DataFrame with price data
    
    Returns:
        dictionary with performance metrics
    """
    # filter transactions for the specific ticker
    ticker_transactions = transactions_df[transactions_df['Ticker'] == ticker].copy()
    ticker_transactions = ticker_transactions.sort_values('Date')

    # get price data for the ticker
    ticker_prices = prices_df[['Date', ticker]].copy()
    ticker_prices = ticker_prices.sort_values('Date')
    
    # calculate shares purchased/sold for each transaction
    ticker_transactions_with_shares = []
    
    for _, txn in ticker_transactions.iterrows():
        # find the closest price date (should be exact match for transaction dates)
        price_on_date = ticker_prices[ticker_prices['Date'] <= txn['Date']].iloc[-1][ticker]
        shares = txn['Amount_USD'] / price_on_date
        
        if txn['Direction'] == 'SELL':
            shares = -shares  # Negative for sales
        
        ticker_transactions_with_shares.append({
            'Date': txn['Date'],
            'Direction': txn['Direction'],
            'Amount_USD': txn['Amount_USD'],
            'Price': price_on_date,
            'Shares': shares
        })
    
    txn_df = pd.DataFrame(ticker_transactions_with_shares)
    
    # 1. Calculate IRR
    # prepare cashflows: negative for purchases, positive for sales
    cashflows = []
    dates = []
    
    for _, txn in txn_df.iterrows():
        if txn['Direction'] == 'BUY':
            cashflows.append(-txn['Amount_USD'])  # negative for outflow
        else:  # SELL
            cashflows.append(txn['Amount_USD'])   # positive for inflow
        dates.append(txn['Date'])
    
    # add final value if there are remaining holdings
    final_shares = txn_df['Shares'].sum()
    if abs(final_shares) > 0.001:  # if there are remaining shares
        final_date = pd.to_datetime('2020-12-31')  # end of analysis period
        final_price = ticker_prices[ticker_prices['Date'] <= final_date].iloc[-1][ticker]
        final_value = final_shares * final_price
        cashflows.append(final_value)
        dates.append(final_date)
    
    try:
        irr = calculate_irr(cashflows, dates)
        irr_annualized = irr
    except:
        irr_annualized = np.nan
    
    # 2. Calculate Time-Weighted Return
    # track portfolio value over time
    daily_positions = []
    current_shares = 0
    
    # create a complete date range
    start_date = ticker_transactions['Date'].min()
    end_date = pd.to_datetime('2020-12-31')
    
    # get daily prices
    daily_prices = ticker_prices[(ticker_prices['Date'] >= start_date) & 
                                (ticker_prices['Date'] <= end_date)].copy()
    
    # calculate daily portfolio values
    for _, price_row in daily_prices.iterrows():
        date = price_row['Date']
        price = price_row[ticker]
        
        # apply any transactions on this date
        day_transactions = txn_df[txn_df['Date'] == date]
        for _, txn in day_transactions.iterrows():
            current_shares += txn['Shares']
        
        portfolio_value = current_shares * price
        daily_positions.append({
            'Date': date,
            'Shares': current_shares,
            'Price': price,
            'Portfolio_Value': portfolio_value
        })
    
    positions_df = pd.DataFrame(daily_positions)
    
    # calculate time-weighted returns
    if len(positions_df) > 1:
        positions_df['Daily_Return'] = positions_df['Portfolio_Value'].pct_change()
        
        # handle days with transactions (exclude from return calculation)
        transaction_dates = set(txn_df['Date'].dt.date)
        positions_df['Transaction_Date'] = positions_df['Date'].dt.date.isin(transaction_dates)
        
        # calculate cumulative return excluding transaction dates
        valid_returns = positions_df[~positions_df['Transaction_Date']]['Daily_Return'].dropna()
        
        if len(valid_returns) > 0:
            cumulative_return = (1 + valid_returns).prod() - 1
            
            # annualize the return
            days_invested = (end_date - start_date).days
            years_invested = days_invested / 365.25
            
            if years_invested > 0:
                annualized_twr = (1 + cumulative_return) ** (1/years_invested) - 1
            else:
                annualized_twr = cumulative_return
        else:
            annualized_twr = 0
    else:
        annualized_twr = 0
    
    # 3. Calculate TVPI (Total Value to Paid-In)
    total_invested = sum(txn['Amount_USD'] for _, txn in txn_df.iterrows() if txn['Direction'] == 'BUY')
    total_divested = sum(txn['Amount_USD'] for _, txn in txn_df.iterrows() if txn['Direction'] == 'SELL')
    
    # current value of remaining holdings
    if abs(final_shares) > 0.001:
        current_value = abs(final_shares) * final_price
    else:
        current_value = 0
    
    total_value = total_divested + current_value
    tvpi = total_value / total_invested if total_invested > 0 else 0
    
    return {
        'ticker': ticker,
        'irr_annualized': irr_annualized,
        'twr_annualized': annualized_twr,
        'tvpi': tvpi,
        'total_invested': total_invested,
        'total_divested': total_divested,
        'current_value': current_value,
        'total_value': total_value,
        'final_shares': final_shares,
        'positions_df': positions_df,
        'transactions_df': txn_df,
        'cashflows': cashflows,
        'cashflow_dates': dates
    }

# calculate metrics for both stocks
aapl_metrics = calculate_performance_metrics('AAPL', transactions_df, prices_df)
tsla_metrics = calculate_performance_metrics('TSLA', transactions_df, prices_df)

print("See calculated IRR, TWR, and TVPI for AAPL and TSLA under PERFORMANCE ANALYTICS SUMMARY at the end of Task b.")

See calculated IRR, TWR, and TVPI for AAPL and TSLA under PERFORMANCE ANALYTICS SUMMARY at the end of Task b.


## Task b: Daily Cashflows and Positions

In [22]:
def generate_daily_cashflows_and_positions(transactions_df, prices_df):
    """
    Generate daily cashflows and portfolio positions (number of shares, market value, percentage allocation of each security in the portfolio) for IRR calculation
    
    returns:
        DataFrame with daily data for both securities
    """
    # create date range from first transaction to end of 2020
    start_date = transactions_df['Date'].min()
    end_date = pd.to_datetime('2020-12-31')
    
    # create daily date range 
    date_range = pd.date_range(start=start_date, end=end_date, freq='D')
    
    daily_data = []
    
    # track positions for each stock
    aapl_shares = 0
    tsla_shares = 0
    
    for date in date_range:
        # check for transactions on this date
        day_transactions = transactions_df[transactions_df['Date'] == date]
        
        # initialize daily values
        aapl_cashflow = 0
        tsla_cashflow = 0
        
        # process transactions
        for _, txn in day_transactions.iterrows():
            # get price for the transaction
            price_data = prices_df[prices_df['Date'] <= date]
            if len(price_data) > 0:
                latest_prices = price_data.iloc[-1]
                
                if txn['Ticker'] == 'AAPL':
                    shares = txn['Amount_USD'] / latest_prices['AAPL']
                    if txn['Direction'] == 'BUY':
                        aapl_cashflow -= txn['Amount_USD']  # purchase
                        aapl_shares += shares
                    else:  # SELL
                        aapl_cashflow += txn['Amount_USD']  # sale
                        aapl_shares -= shares
                        
                elif txn['Ticker'] == 'TSLA':
                    shares = txn['Amount_USD'] / latest_prices['TSLA']
                    if txn['Direction'] == 'BUY':
                        tsla_cashflow -= txn['Amount_USD']  # purchase
                        tsla_shares += shares
                    else:  # SELL
                        tsla_cashflow += txn['Amount_USD']  #  sale
                        tsla_shares -= shares
        
        # get current prices for position valuation
        price_data = prices_df[prices_df['Date'] <= date]
        if len(price_data) > 0:
            latest_prices = price_data.iloc[-1]
            aapl_position_value = aapl_shares * latest_prices['AAPL']
            tsla_position_value = tsla_shares * latest_prices['TSLA']
        else:
            aapl_position_value = 0
            tsla_position_value = 0
        
        daily_data.append({
            'Date': date,
            'AAPL_Cashflow': aapl_cashflow,
            'TSLA_Cashflow': tsla_cashflow,
            'AAPL_Shares': aapl_shares,
            'TSLA_Shares': tsla_shares,
            'AAPL_Position_Value': aapl_position_value,
            'TSLA_Position_Value': tsla_position_value,
            'Total_Portfolio_Value': aapl_position_value + tsla_position_value
        })
    
    return pd.DataFrame(daily_data)

# generate daily data
daily_portfolio = generate_daily_cashflows_and_positions(transactions_df, prices_df)

# print("Original transactions:")
# print(transactions_df)
# print(f"\nTransaction amounts: {transactions_df['Amount_USD'].tolist()}")
# print(f"Transaction dates: {transactions_df['Date'].tolist()}")

# Check if amounts are in thousands
# print("\nprice data:")
# jan_2011_prices = prices_df[(prices_df['Date'] >= '2011-01-01') & (prices_df['Date'] <= '2011-01-31')]
# print(jan_2011_prices.head())

print("\n=== DAILY CASHFLOWS AND POSITIONS ===")
print("First 30 days of daily data:")
print("\nColumns: Date | AAPL_Cashflow | TSLA_Cashflow | AAPL_Shares | TSLA_Shares | Portfolio_Value")
print("-" * 100)

# Show first 30 days of daily data
daily_sample = daily_portfolio.head(30)
for _, row in daily_sample.iterrows():
    print(f"{row['Date'].strftime('%Y-%m-%d')} | "
          f"${row['AAPL_Cashflow']:>10,.0f} | "
          f"${row['TSLA_Cashflow']:>10,.0f} | "
          f"{row['AAPL_Shares']:>10,.0f} | "
          f"{row['TSLA_Shares']:>10,.0f} | "
          f"${row['Total_Portfolio_Value']:>12,.0f}")

print(f"\n... (showing 30 of {len(daily_portfolio)} total daily records)")

# ahow transaction dates specifically
print("\n=== TRANSACTION DATES (Non-zero cashflows) ===")
transaction_days = daily_portfolio[
    (daily_portfolio['AAPL_Cashflow'] != 0) | 
    (daily_portfolio['TSLA_Cashflow'] != 0)
]
print("Date | AAPL_Cashflow | TSLA_Cashflow | AAPL_Shares | TSLA_Shares | Portfolio_Value")
print("-" * 85)
for _, row in transaction_days.iterrows():
    print(f"{row['Date'].strftime('%Y-%m-%d')} | "
          f"${row['AAPL_Cashflow']:>10,.0f} | "
          f"${row['TSLA_Cashflow']:>10,.0f} | "
          f"{row['AAPL_Shares']:>10,.0f} | "
          f"{row['TSLA_Shares']:>10,.0f} | "
          f"${row['Total_Portfolio_Value']:>12,.0f}")

# print(f"\nTotal daily records generated: {len(daily_portfolio)}")
# print(f"Date range: {daily_portfolio['Date'].min().strftime('%Y-%m-%d')} to {daily_portfolio['Date'].max().strftime('%Y-%m-%d')}")

# daily data for IRR calculation
# print("- Daily cashflows for each security (negative for purchases, positive for sales)")
# print("- Daily position holdings (number of shares)")
# print("- Daily position values (shares × current price)")
# print("- Total portfolio value evolution")

# Summary statistics
print(f"\nDaily Data Summary:")
print(f"- Total transaction days: {len(transaction_days)}")
print(f"- Days with non-zero AAPL cashflow: {len(daily_portfolio[daily_portfolio['AAPL_Cashflow'] != 0])}")
print(f"- Days with non-zero TSLA cashflow: {len(daily_portfolio[daily_portfolio['TSLA_Cashflow'] != 0])}")
print(f"- Portfolio start value: ${daily_portfolio.iloc[0]['Total_Portfolio_Value']:,.0f}")
print(f"- Portfolio end value: ${daily_portfolio.iloc[-1]['Total_Portfolio_Value']:,.0f}")

# =============================================================================
# RESULTS SUMMARY
# =============================================================================

print("\n" + "="*80)
print("PERFORMANCE ANALYTICS SUMMARY")
print("="*80)

print(f"\nAAPL Performance Metrics:")
print(f"  IRR (Annualized):           {aapl_metrics['irr_annualized']:.2%}")
print(f"  Time-Weighted Return:       {aapl_metrics['twr_annualized']:.2%}")
print(f"  TVPI:                       {aapl_metrics['tvpi']:.2f}x")
print(f"  Total Invested:             ${aapl_metrics['total_invested']:,.0f}")
print(f"  Total Value:                ${aapl_metrics['total_value']:,.0f}")
print(f"  Current Holdings:           {aapl_metrics['final_shares']:.0f} shares")

print(f"\nTSLA Performance Metrics:")
print(f"  IRR (Annualized):           {tsla_metrics['irr_annualized']:.2%}")
print(f"  Time-Weighted Return:       {tsla_metrics['twr_annualized']:.2%}")
print(f"  TVPI:                       {tsla_metrics['tvpi']:.2f}x")
print(f"  Total Invested:             ${tsla_metrics['total_invested']:,.0f}")
print(f"  Total Value:                ${tsla_metrics['total_value']:,.0f}")
print(f"  Current Holdings:           {tsla_metrics['final_shares']:.0f} shares")



=== DAILY CASHFLOWS AND POSITIONS ===
First 30 days of daily data:

Columns: Date | AAPL_Cashflow | TSLA_Cashflow | AAPL_Shares | TSLA_Shares | Portfolio_Value
----------------------------------------------------------------------------------------------------
2011-01-03 | $-1,000,000 | $-1,000,000 |     84,959 |    187,829 | $   2,000,000
2011-01-04 | $         0 | $         0 |     84,959 |    187,829 | $   2,007,097
2011-01-05 | $         0 | $         0 |     84,959 |    187,829 | $   2,021,330
2011-01-06 | $         0 | $         0 |     84,959 |    187,829 | $   2,059,955
2011-01-07 | $         0 | $         0 |     84,959 |    187,829 | $   2,080,731
2011-01-08 | $         0 | $         0 |     84,959 |    187,829 | $   2,080,731
2011-01-09 | $         0 | $         0 |     84,959 |    187,829 | $   2,080,731
2011-01-10 | $         0 | $         0 |     84,959 |    187,829 | $   2,107,841
2011-01-11 | $         0 | $         0 |     84,959 |    187,829 | $   2,049,396
2011-01-1

## Task c: Performance Analysis and Conclusions

In [27]:
print("\n1. COMPARATIVE PERFORMANCE:")
print(f"   • AAPL IRR: {aapl_metrics['irr_annualized']:.2%} vs TSLA IRR: {tsla_metrics['irr_annualized']:.2%}")
print(f"   • AAPL TWR: {aapl_metrics['twr_annualized']:.2%} vs TSLA TWR: {tsla_metrics['twr_annualized']:.2%}")
print(f"   • AAPL TVPI: {aapl_metrics['tvpi']:.2f}x vs TSLA TVPI: {tsla_metrics['tvpi']:.2f}x")

print("\n2. INVESTMENT TIMING ANALYSIS:")
print("   AAPL Transactions:")
for _, txn in aapl_metrics['transactions_df'].iterrows():
    print(f"   • {txn['Date'].strftime('%Y-%m-%d')}: {txn['Direction']} at ${txn['Price']:.2f}/share")

print("   TSLA Transactions:")
for _, txn in tsla_metrics['transactions_df'].iterrows():
    print(f"   • {txn['Date'].strftime('%Y-%m-%d')}: {txn['Direction']} at ${txn['Price']:.2f}/share")

print("\n3. KEY OBSERVATIONS:")

# Performance comparison
if tsla_metrics['irr_annualized'] > aapl_metrics['irr_annualized']:
    better_irr = "TSLA"
    irr_diff = tsla_metrics['irr_annualized'] - aapl_metrics['irr_annualized']
else:
    better_irr = "AAPL"
    irr_diff = aapl_metrics['irr_annualized'] - tsla_metrics['irr_annualized']

print(f"   • {better_irr} delivered superior returns with {irr_diff:.2%} higher IRR")

# Risk assessment based on transaction timing
print("   • Manager demonstrated excellent market timing on TSLA:")
print("     - Bought early (2011) when TSLA was around $5.32")
print("     - Sold strategically in April 2019 and March 2020")
print("     - Captured significant price appreciation before major volatility")

print("   • AAPL investment showed more conservative approach:")
print("     - Consistent buying in 2011-2012 period")
print("     - Held position through entire analysis period")
print("     - Benefited from steady long-term appreciation")

# Portfolio allocation
total_invested = aapl_metrics['total_invested'] + tsla_metrics['total_invested']
aapl_allocation = aapl_metrics['total_invested'] / total_invested
tsla_allocation = tsla_metrics['total_invested'] / total_invested

print(f"\n4. PORTFOLIO ALLOCATION:")
print(f"   • AAPL: {aapl_allocation:.1%} of total investment (${aapl_metrics['total_invested']:,.0f})")
print(f"   • TSLA: {tsla_allocation:.1%} of total investment (${tsla_metrics['total_invested']:,.0f})")

print(f"\n5. OVERALL ASSESSMENT:")
print(f"   • Combined portfolio performance demonstrates strong stock selection")
print(f"   • Manager showed ability to time exits effectively (TSLA sales)")
print(f"   • Diversified approach with different holding strategies per stock")
print(f"   • Both investments significantly outperformed risk-free rates")

# Calculate portfolio-level metrics
portfolio_total_invested = aapl_metrics['total_invested'] + tsla_metrics['total_invested']
portfolio_total_value = aapl_metrics['total_value'] + tsla_metrics['total_value']
portfolio_tvpi = portfolio_total_value / portfolio_total_invested

print(f"\n6. PORTFOLIO-LEVEL METRICS:")
print(f"   • Total Capital Invested: ${portfolio_total_invested:,.0f}")
print(f"   • Total Portfolio Value: ${portfolio_total_value:,.0f}")
print(f"   • Portfolio TVPI: {portfolio_tvpi:.2f}x")
print(f"   • Absolute Profit: ${portfolio_total_value - portfolio_total_invested:,.0f}")

print("\n" + "="*80)


1. COMPARATIVE PERFORMANCE:
   • AAPL IRR: 27.55% vs TSLA IRR: 37.93%
   • AAPL TWR: 27.24% vs TSLA TWR: 62.28%
   • AAPL TVPI: 10.15x vs TSLA TVPI: 17.49x

2. INVESTMENT TIMING ANALYSIS:
   AAPL Transactions:
   • 2011-01-03: BUY at $11.77/share
   • 2012-01-03: BUY at $14.69/share
   TSLA Transactions:
   • 2011-01-03: BUY at $5.32/share
   • 2019-04-01: SELL at $57.84/share
   • 2020-03-23: SELL at $86.86/share

3. KEY OBSERVATIONS:
   • TSLA delivered superior returns with 10.37% higher IRR
   • Manager demonstrated excellent market timing on TSLA:
     - Bought early (2011) when TSLA was around $5.32
     - Sold strategically in April 2019 and March 2020
     - Captured significant price appreciation before major volatility
   • AAPL investment showed more conservative approach:
     - Consistent buying in 2011-2012 period
     - Held position through entire analysis period
     - Benefited from steady long-term appreciation

4. PORTFOLIO ALLOCATION:
   • AAPL: 66.7% of total inv